In [1]:
import os
import math
import pandas as pd
import numpy as np

# Configuration settings
from chasm.config import CONFIG

# PLINK preprocessing utilities
from chasm.plink_preprocessing import (
    test, 
    concat_AFs, 
    divide_into_chunks, 
    make_ids,
    is_snp
)

from chasm.gwas import make_AFs, ols_regression




In [2]:
# Changing the GTM data into a pickle file having an id file and calculating the AFs for each SNP 
# with the CHROM:POS for every corresponding RSID
"""
# GTM data
path_data_raw = f"{CONFIG['PATH_data']}/00_raw/recoded_1000G.raw"
path_data_ids = f"{CONFIG['PATH_data']}/00_raw/recoded_1000G.raw.noadmixed.ids"
path_data_lbls3 = f"{CONFIG['PATH_data']}/00_raw/recoded_1000G.raw.noadmixed.lbls3"

geno = pd.read_csv(path_data_raw, sep=" ")
non_snp_cols = [col for col in geno.columns if not is_snp(col)]
path_usefull = f"{CONFIG['PATH_data']}/usefull/"
os.makedirs(path_usefull, exist_ok=True)
geno[non_snp_cols].to_pickle(f"{path_usefull}/ids.pkl")
geno = geno.drop(columns=non_snp_cols)
path_raw = f"{CONFIG['PATH_data']}/01_raw/"
os.makedirs(path_raw, exist_ok=True)
geno.to_pickle(f"{path_raw}/geno.pkl")
allele_frequencies = geno.sum(axis=0) / (2 * geno.shape[0])

allele_frequencies_df = pd.DataFrame(allele_frequencies, columns=["AF"])
# Assuming allele_frequencies is a Pandas Series
allele_frequencies_df = allele_frequencies.reset_index()
allele_frequencies_df.columns = ["snp_rs", "AF"]  # Rename columns
# Create a new column without the allele suffix
allele_frequencies_df['RSID'] = allele_frequencies_df['snp_rs'].str.replace(r'_[ACTG]$', '', regex=True)

merged_dfs = []
path_ensembl = f"/mnt/e/1000G_data/usefull/ensembl_build"
for chrom in list(range(22)):
    chrom += 1
    path_ensembl_chrom = f"{path_ensembl}/chrom_{chrom}"
    for build in [f for f in os.listdir(path_ensembl_chrom) if f.startswith('build')]:
        build = pd.read_pickle(f"{path_ensembl_chrom}/{build}")
        build['CHROM'] = chrom
        merged_dfs.append(pd.merge(allele_frequencies_df, build, left_on='RSID', right_on='RSID', how='inner'))
        
df = pd.concat(merged_dfs, axis=0)
# Sorting by CHROM and then POS
df_sorted = df.sort_values(by=["CHROM", "POS"]).reset_index(drop=True)
df_sorted.to_pickle(f"{path_usefull}/allele_frequencies.pkl")
"""

'\n# GTM data\npath_data_raw = f"{CONFIG[\'PATH_data\']}/00_raw/recoded_1000G.raw"\npath_data_ids = f"{CONFIG[\'PATH_data\']}/00_raw/recoded_1000G.raw.noadmixed.ids"\npath_data_lbls3 = f"{CONFIG[\'PATH_data\']}/00_raw/recoded_1000G.raw.noadmixed.lbls3"\n\ngeno = pd.read_csv(path_data_raw, sep=" ")\nnon_snp_cols = [col for col in geno.columns if not is_snp(col)]\npath_usefull = f"{CONFIG[\'PATH_data\']}/usefull/"\nos.makedirs(path_usefull, exist_ok=True)\ngeno[non_snp_cols].to_pickle(f"{path_usefull}/ids.pkl")\ngeno = geno.drop(columns=non_snp_cols)\npath_raw = f"{CONFIG[\'PATH_data\']}/01_raw/"\nos.makedirs(path_raw, exist_ok=True)\ngeno.to_pickle(f"{path_raw}/geno.pkl")\nallele_frequencies = geno.sum(axis=0) / (2 * geno.shape[0])\n\nallele_frequencies_df = pd.DataFrame(allele_frequencies, columns=["AF"])\n# Assuming allele_frequencies is a Pandas Series\nallele_frequencies_df = allele_frequencies.reset_index()\nallele_frequencies_df.columns = ["snp_rs", "AF"]  # Rename columns\n# Crea

In [3]:
# Divide the AFs into chunks
"""
size_chunck = 20_000
min_maf = 0.01

path_raw = f"{CONFIG['PATH_data']}/01_raw/"
geno = pd.read_pickle(f"{path_raw}/geno.pkl")
geno.fillna(0, inplace=True)
geno = (geno - 1)*-1

path_afs = f"{CONFIG['PATH_data']}/usefull/allele_frequencies.pkl"
path_output = f"{CONFIG['PATH_data']}/03_macro_similar_AF/"
os.makedirs(path_output, exist_ok=True)
afs = pd.read_pickle(path_afs)

for chrom in afs['CHROM'].unique():
    path_output_chrom = f"{path_output}/chrom_{chrom}/"
    os.makedirs(path_output_chrom, exist_ok=True)
    
    afs_chrom = afs[afs['CHROM'] == chrom]
    afs_chrom = afs_chrom[afs_chrom['AF'] > min_maf]

    afs_chrom = afs_chrom.sort_values(by=["AF"], ascending=False).reset_index(drop=True)
    nr_snps_total = len(afs_chrom)
    num_subframes = nr_snps_total//size_chunck
    remaining_rows = nr_snps_total%size_chunck
    
    try:
        to_divide_in = remaining_rows//num_subframes
        rest = nr_snps_total- ((size_chunck + to_divide_in)*num_subframes)
        snps_per_segments = np.ones(num_subframes) * (size_chunck+ to_divide_in)
        to_add = np.concatenate((np.ones(rest), np.zeros(num_subframes - rest)),axis = 0)
        snps_per_segments = snps_per_segments + to_add
    except Exception as e:
        snps_per_segments = [nr_snps_total]
        
    # Make Chunks per chromosomes
    start = 0
    end = 0
    i = 0
    for nr_snps in snps_per_segments:
        end = int(end + nr_snps)
        AF_chunk = afs_chrom[start:end].copy()
        start = int(start + nr_snps)
        feature_size = AF_chunk.shape[0]
        i+=1
        minaf = np.round(AF_chunk['AF'].min(),2)
        maxaf = np.round(AF_chunk['AF'].max(),2)
        geno[AF_chunk['snp_rs']].to_pickle(f"{path_output_chrom}/chunk_{i}_size_{len(AF_chunk)}_mafs_{minaf}_{maxaf}.pkl")
"""

'\nsize_chunck = 20_000\nmin_maf = 0.01\n\npath_raw = f"{CONFIG[\'PATH_data\']}/01_raw/"\ngeno = pd.read_pickle(f"{path_raw}/geno.pkl")\ngeno.fillna(0, inplace=True)\ngeno = (geno - 1)*-1\n\npath_afs = f"{CONFIG[\'PATH_data\']}/usefull/allele_frequencies.pkl"\npath_output = f"{CONFIG[\'PATH_data\']}/03_macro_similar_AF/"\nos.makedirs(path_output, exist_ok=True)\nafs = pd.read_pickle(path_afs)\n\nfor chrom in afs[\'CHROM\'].unique():\n    path_output_chrom = f"{path_output}/chrom_{chrom}/"\n    os.makedirs(path_output_chrom, exist_ok=True)\n    \n    afs_chrom = afs[afs[\'CHROM\'] == chrom]\n    afs_chrom = afs_chrom[afs_chrom[\'AF\'] > min_maf]\n\n    afs_chrom = afs_chrom.sort_values(by=["AF"], ascending=False).reset_index(drop=True)\n    nr_snps_total = len(afs_chrom)\n    num_subframes = nr_snps_total//size_chunck\n    remaining_rows = nr_snps_total%size_chunck\n    \n    try:\n        to_divide_in = remaining_rows//num_subframes\n        rest = nr_snps_total- ((size_chunck + to_div

# Choose SNPs to project on 1 DIM

In [4]:
# 1 create PCA

nr_snps_for_PCA = 20_000

path_input = f"{CONFIG['PATH_data']}/03_macro_similar_AF/"

chroms = os.listdir(path_input)
nr_snps_for_PCA_per_chrom = math.ceil(nr_snps_for_PCA/len(chroms))
genos = []
for chrom in chroms:
    path_chrom = f"{path_input}/{chrom}"
    chunks = os.listdir(path_chrom)
    nr_snps_for_PCA_per_chunks = math.ceil(nr_snps_for_PCA_per_chrom / len(chunks))

    for chunk in chunks:
        path_chunk = f"{path_chrom}/{chunk}"
        geno = pd.read_pickle(path_chunk)
        # Get number of available columns
        num_available_columns = geno.shape[1]

        # Adjust n if needed
        n = min(nr_snps_for_PCA_per_chunks, num_available_columns)
        geno = geno.sample(n=n, axis=1)
        genos.append(geno)
genos = pd.concat(genos, axis=1)


In [5]:
genos

,rs4424509_G,rs67665173_A,rs834340_T,rs10776705_C,rs11165999_C,rs72855780_T,rs1891143_G,rs6702870_T,rs10915960_A,rs9427716_T,...,rs8138285_G,rs134178_G,rs6002725_C,rs3859830_A,rs5998198_G,rs139760_A,rs6519496_T,rs1297382_T,rs75860721_C,rs5764760_A
0,0,1,0,1,-1,1,0,1,0,0,...,0,0,0,-1,-1,0,-1,1,1,1
1,1,1,1,0,-1,0,0,0,-1,0,...,0,-1,0,1,0,1,0,1,1,-1
2,-1,1,1,0,0,1,-1,1,-1,1,...,0,-1,0,-1,0,0,-1,-1,-1,0
3,1,1,1,1,-1,1,-1,-1,-1,0,...,1,0,-1,1,0,-1,1,1,1,1
4,-1,1,1,0,1,1,0,0,0,0,...,0,-1,1,0,1,-1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,1,1,1,1,1,0,0,0,1,0,...,1,1,1,-1,0,0,1,0,1,-1
2500,0,1,1,0,-1,1,-1,1,0,-1,...,0,0,1,1,1,1,0,-1,1,0
2501,0,1,1,0,0,1,0,1,0,0,...,1,0,0,0,0,0,-1,1,-1,-1
2502,0,0,0,0,-1,0,0,1,0,0,...,0,0,-1,-1,0,1,1,0,0,1


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize the data (zero mean, unit variance)
scaler = StandardScaler()
genos = scaler.fit_transform(genos)  # Returns a NumPy array

# Apply PCA
n_components = 5  # Number of principal components to keep
pca = PCA(n_components=n_components)
genos_pca = pca.fit_transform(genos)  # Transform the data

# Convert PCA output to DataFrame
genos_pca = pd.DataFrame(genos_pca, columns=[f'PC{i+1}' for i in range(n_components)])

In [7]:
genos_pca

,PC1,PC2,PC3,PC4,PC5
0,28.590115,3.941087,1.285033,-1.113609,-0.651692
1,15.428179,2.918643,-8.906655,4.175352,-0.782347
2,33.430092,6.913488,1.452569,-1.261835,-1.964900
3,27.256914,3.643712,1.499485,-5.127852,-1.271657
4,32.874977,7.338159,1.842780,-1.103001,-0.276763
...,...,...,...,...,...
2499,-8.633141,-7.279657,-25.850437,10.981157,0.788396
2500,-10.608437,-9.350436,-26.211096,8.627377,-1.844377
2501,-9.097981,-5.635283,-24.906864,8.474552,-2.207233
2502,-9.475221,-7.707817,-26.470612,8.820381,-0.976369


In [29]:
path_input = f"{CONFIG['PATH_data']}/03_macro_similar_AF/"
chroms = [f for f in os.listdir(path_input) if f.startswith('chrom')]
p_vals = []
betas = []
snps = []
for chrom in chroms:
    path_chrom = f"{path_input}/{chrom}"
    chunks = os.listdir(path_chrom)
    for chunk in chunks:
        path_chunk = f"{path_chrom}/{chunk}"
        geno = pd.read_pickle(path_chunk)
        for snp in geno.columns:
            [beta_values, p_values] = ols_regression(genos_pca['PC1'], geno[snp], covs=None)
            p_vals.append(p_values[snp])
            betas.append(beta_values[snp])
            snps.append(snp)
            

p_vals = pd.DataFrame(data = {'pval': p_vals, 'betas':betas, 'snp_rs':snps})
p_vals['-logp'] = -np.log10(p_vals['pval'])


/home/mat2m10/.pyenv/versions/3.8.12/envs/pca_projection/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [30]:
bonfer = 8.3
to_keep = p_vals[p_vals['-logp'] > bonfer]
other = p_vals[p_vals['-logp'] <= bonfer]

In [33]:
pos = pd.read_pickle(f"{CONFIG['PATH_data']}/usefull/allele_frequencies.pkl")

In [35]:
to_keep = pd.merge(to_keep, pos, left_on='snp_rs', right_on='snp_rs', how='inner')
other = pd.merge(other, pos, left_on='snp_rs', right_on='snp_rs', how='inner')

In [36]:
to_keep

,pval,betas,snp_rs,-logp,AF,RSID,POS,CHROM
0,1.641501e-64,9.543889,rs3928804_T,63.784759,0.500000,rs3928804,160943517,1
1,8.059506e-24,5.723635,rs12745158_G,23.093692,0.500000,rs12745158,39802413,1
2,1.092316e-21,-5.472473,rs703115_C,20.961652,0.500000,rs703115,158673247,1
3,1.854242e-09,3.489105,rs10888929_T,8.731834,0.500000,rs10888929,39681671,1
4,3.042075e-13,4.139335,rs814346_A,12.516830,0.500000,rs814346,190795603,1
...,...,...,...,...,...,...,...,...
21349,5.053940e-20,11.640626,rs2013215_C,19.296370,0.055911,rs2013215,49447222,22
21350,2.773652e-50,-18.127401,rs6004890_G,49.556948,0.055511,rs6004890,26011025,22
21351,1.676378e-17,-11.429114,rs117767918_A,16.775628,0.051118,rs117767918,26721111,22
21352,3.996715e-09,-8.010530,rs12167043_A,8.398297,0.050919,rs12167043,17286193,22


In [37]:
other

,pval,betas,snp_rs,-logp,AF,RSID,POS,CHROM
0,0.001380,-1.850217,rs6695244_A,2.860106,0.500000,rs6695244,18407223,1
1,0.481211,0.413966,rs3010023_A,0.317665,0.500000,rs3010023,212042477,1
2,0.147103,-0.847900,rs7516288_A,0.832378,0.500000,rs7516288,22320295,1
3,0.010445,-1.461384,rs6704281_T,1.981080,0.500000,rs6704281,221334304,1
4,0.007284,1.557526,rs10801960_A,2.137619,0.500000,rs10801960,117483702,1
...,...,...,...,...,...,...,...,...
17585,0.013558,3.196345,rs543164958_G,1.867794,0.057508,rs543164958,17511736,22
17586,0.000003,-5.930929,rs113570940_C,5.479788,0.055711,rs113570940,19545170,22
17587,0.302625,-1.339838,rs377221550_T,0.519095,0.055511,rs377221550,15611674,22
17588,0.078167,-2.233307,rs3984495_A,1.106978,0.054712,rs3984495,18743311,22
